In [44]:
import datetime as dt

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from creme import (
    optim,
    linear_model,
    preprocessing,
    tree,
    compose,
    metrics,
    stream,
    compat,
    model_selection,
    ensemble,
)

%matplotlib inline

In [45]:
ignored_fields = [
    'gameDuration',
]

In [46]:
params = dict(
    target_name='hasWon',
    converters={
        'hasWon': int,
        'goldDiff': int,
        'expDiff': int,
        'champLevelDiff': float,
        'isFirstTower': int,
        'isFirstBlood': int,
        'killedFireDrake': int,
        'killedWaterDrake': int,
        'killedAirDrake': int,
        'killedEarthDrake': int,
        'killedElderDrake': int,
        'lostFireDrake': int,
        'lostWaterDrake': int,
        'lostAirDrake': int,
        'lostEarthDrake': int,
        'lostElderDrake': int,
        'killedBaronNashor': int,
        'lostBaronNashor': int,
        'killedRiftHerald': int,
        'lostRiftHerald': int,
        'destroyedTopInhibitor': int,
        'destroyedMidInhibitor': int,
        'destroyedBotInhibitor': int,
        'lostTopInhibitor': int,
        'lostMidInhibitor': int,
        'lostBotInhibitor': int,
        'destroyedTopNexusTurret': int,
        'destroyedMidNexusTurret': int,
        'destroyedBotNexusTurret': int,
        'lostTopNexusTurret': int,
        'lostMidNexusTurret': int,
        'lostBotNexusTurret': int,
        'destroyedTopBaseTurret': int,
        'destroyedMidBaseTurret': int,
        'destroyedBotBaseTurret': int,
        'lostTopBaseTurret': int,
        'lostMidBaseTurret': int,
        'lostBotBaseTurret': int,
        'destroyedTopInnerTurret': int,
        'destroyedMidInnerTurret': int,
        'destroyedBotInnerTurret': int,
        'lostTopInnerTurret': int,
        'lostMidInnerTurret': int,
        'lostBotInnerTurret': int,
        'destroyedTopOuterTurret': int,
        'destroyedMidOuterTurret': int,
        'destroyedBotOuterTurret': int,
        'lostTopOuterTurret': int,
        'lostMidOuterTurret': int,
        'lostBotOuterTurret': int,
        'kills': int,
        'deaths': int,
        'assists': int,
        'wardsPlaced': int,
        'wardsDestroyed': int,
        'wardsLost': int,
    },
    drop=ignored_fields,
)

In [47]:
X_y = stream.iter_csv('lol_ranked_games.csv', **params)

In [48]:
opt = optim.SGD(lr=0.1)

In [49]:
model = compose.Discard('gameId', 'frame')
model |= compose.Discard('isFirstTower', 'isFirstBlood') + preprocessing.StandardScaler()
# model |= preprocessing.OneHotEncoder('isFirstBlood', sparse=True)
model |= linear_model.LogisticRegression(opt)

model

Pipeline (
  Discard (
    frame
    gameId
  ),
  TransformerUnion (
    Discard (
      isFirstBlood
      isFirstTower
    ),
    StandardScaler (
      with_mean=True
      with_std=True
    )
  ),
  LogisticRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.1
      )
    )
    loss=Log ()
    l2=0.
    intercept=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

In [50]:
metrics = [
    metrics.Accuracy(),
    metrics.Precision(),
    metrics.Recall(),
    metrics.F1(),
    metrics.ROCAUC(),
    metrics.ConfusionMatrix(),
]

In [51]:
def update_metrics(y_true=None, y_pred=None):
    if metrics is None:
        raise ValueError('No metrics have been defined.')
        
    if y_true is None or y_pred is None:
        raise ValueError('Incorrect y values.')
            
    for metric in metrics:
        metric.update(y_true, y_pred)

In [52]:
def display_metrics():
    if metrics is None:
        raise ValueError('No metrics have been defined.')
        
    for metric in metrics:
        print(metric)

In [53]:
for x, y in X_y:
    model.debug_one(x)
    break

0. Input
--------
assists: 5 (int)
champLevelDiff: -0.20000 (float)
deaths: 7 (int)
destroyedBotBaseTurret: 0 (int)
destroyedBotInhibitor: 0 (int)
destroyedBotInnerTurret: 0 (int)
destroyedBotNexusTurret: 0 (int)
destroyedBotOuterTurret: 0 (int)
destroyedMidBaseTurret: 0 (int)
destroyedMidInhibitor: 0 (int)
destroyedMidInnerTurret: 0 (int)
destroyedMidNexusTurret: 0 (int)
destroyedMidOuterTurret: 0 (int)
destroyedTopBaseTurret: 0 (int)
destroyedTopInhibitor: 0 (int)
destroyedTopInnerTurret: 0 (int)
destroyedTopNexusTurret: 0 (int)
destroyedTopOuterTurret: 0 (int)
expDiff: -147 (int)
frame: 10 (str)
gameId: 4546233126 (str)
goldDiff: -448 (int)
isFirstBlood: 1 (int)
isFirstTower: 0 (int)
killedAirDrake: 0 (int)
killedBaronNashor: 0 (int)
killedEarthDrake: 1 (int)
killedElderDrake: 0 (int)
killedFireDrake: 0 (int)
killedRiftHerald: 0 (int)
killedWaterDrake: 0 (int)
kills: 4 (int)
lostAirDrake: 0 (int)
lostBaronNashor: 0 (int)
lostBotBaseTurret: 0 (int)
lostBotInhibitor: 0 (int)
lostBotIn

In [55]:
idx = 0
for x, y in X_y:
    y_hat = model.predict_one(x)
    model.fit_one(x, y)
    
    update_metrics(y, y_hat)
    
    if idx != 0 and idx % 50_000 == 0:
        display_metrics()
    idx += 1

Accuracy: 84.98%
Precision: 0.848996
Recall: 0.846764
F1: 0.847878
ROCAUC: 0.84975
             False        1
        0    21558     3723
        1     3788    20932
Accuracy: 85.08%
Precision: 0.850549
Recall: 0.848527
F1: 0.849537
ROCAUC: 0.850824
             False        1
        0    42976     7399
        1     7517    42109
Accuracy: 85.06%
Precision: 0.849618
Recall: 0.848887
F1: 0.849253
ROCAUC: 0.85056
             False        1
        0    64451    11175
        1    11239    63136
Accuracy: 84.98%
Precision: 0.849818
Recall: 0.848248
F1: 0.849032
ROCAUC: 0.849784
             False        1
        0    85482    14929
        1    15113    84477


In [56]:
X_y_test = stream.iter_csv('stats_diam.csv', **params)

In [ ]:
last_game_id = None

for x, y in X_y_test:
    y_hat = model.predict_one(x)
    print(f"Game: {x['gameId']} - Frame #{x['frame']} - Target: {y} - Predict: {model.predict_proba_one(x)}")
    model.fit_one(x, y)
    update_metrics(y, y_hat)
    
    if last_game_id is not None and last_game_id != x['gameId']:
        display_metrics()
        
    last_game_id = x['gameId']